In [1]:
#Import bibliotek
import mysql.connector as sql
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import numpy as np

#Połączenie z bazą danych
username = 'nick'
password = 'haslo'
host = 'localhost'
database = 'airlines'
port = port

#Tworzenie połączenia z bazą
url = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = sqlalchemy.create_engine(url)

#Implementacja read_sql_table
def read_sql_table(table):
    df = pd.read_sql(f"SELECT * FROM {table}", url)
    return df



#Zaczytanie ramki flight_df
flight_df = pd.read_csv(r"../data/processed/flight_df_02.csv")

flight_df.info()

#Wczytanie ramki danych
airport_list_df = read_sql_table('airport_list')

#Sprawdzenie, czy występują duplikaty
is_unique = airport_list_df['origin_airport_id'].is_unique
is_unique

#Złączenie tabel
flight_df = pd.merge(flight_df, airport_list_df[['origin_airport_id', 'origin_city_name']],
                     on='origin_airport_id', how='left')

# Złączenie tabel
flight_df = pd.merge(flight_df,
                     airport_list_df[['origin_airport_id', 'origin_city_name']],
                     left_on='dest_airport_id',
                     right_on='origin_airport_id',
                     how='left')

# Przemianowanie kolumny origin_city_name na destination_city_name
flight_df.rename(columns={'origin_city_name_y': 'destination_city_name'}, inplace=True)
# Usunięcie zbędnej kolumny airport_id
flight_df.drop(columns=['origin_airport_id_y'], inplace=True)

#Zmiana nazw kolumn
flight_df = flight_df.rename(columns={'origin_city_name_x': 'origin_city_name'})
flight_df = flight_df.rename(columns={'origin_airport_id_x': 'origin_airport_id'})
flight_df



# Analiza według lotnisk oraz tras

#Wyznaczenie ramki `top_airports_origin_df`

#Wiersze, gdzie lot nie jest anulowany:
not_delayed = flight_df[flight_df['is_delayed'] == False]
not_delayed.info()

#Grupowanie danych według lotnisk wylotu i zliczanie liczby lotów
top_airports_origin_df = not_delayed.groupby('origin_airport_id').size().reset_index(name='number_of_flights')

#Sortowanie według liczby lotów malejąco
top_airports_origin_df = top_airports_origin_df.sort_values(by='number_of_flights', ascending=False)

# Grupowanie danych według lotnisk przylotu i zliczanie liczby lotów
top_airports_destination_df = not_delayed.groupby('dest_airport_id').size().reset_index(name='number_of_flights')

# Sortowanie według liczby lotów malejąco
top_airports_destination_df = top_airports_destination_df.sort_values(by='number_of_flights', ascending=False)
top_airports_destination_df.head(10)



#Wyznaczenie ramki top_route_df

# Tworzenie kolumny 'route' zawierającej połączenia lotnisk w formie krotki
not_delayed['route'] = list(zip(not_delayed['origin_airport_id'], not_delayed['dest_airport_id']))

# Grupowanie danych według tras (połączeń lotnisk) i zliczanie liczby lotów na każdej trasie
top_route_df = not_delayed.groupby('route').size().reset_index(name='number_of_flights')

# Sortowanie według liczby lotów malejąco
top_route_df = top_route_df.sort_values(by='number_of_flights', ascending=False)



#Wzbogacenie o dane pogodowe

#Wczytanie ramki airport_weather
airport_weather_df = read_sql_table('airport_weather')
airport_weather_df

#Oczyszczenie z nadmiarowych kolumn
airport_weather_df = airport_weather_df[['station', 'name', 'date', 'prcp', 'snow', 'snwd', 'tmax', 'awnd']]
airport_weather_df

#Wykonanie łączenia danych za pomocą inner joina
airport_weather_df = airport_weather_df.merge(airport_list_df[['name', 'origin_airport_id']], how='inner', left_on='name', right_on='name')

#Usunięcie brakujących wartości
airport_weather_df = airport_weather_df.dropna(subset=['name'])
airport_weather_df

#Zmiana date na DATETIME
airport_weather_df['date'] = pd.to_datetime(airport_weather_df['date'])

#Połączenie kolumn month, day_of_month i year w nową kolumnę date w formacie 'YYYY-MM-DD'
flight_df['date'] = pd.to_datetime(flight_df[['year', 'month', 'day_of_month']].astype(str).agg('-'.join, axis=1))

#Złączenie tabel
flight_df = flight_df.merge(airport_weather_df, on=['date', 'origin_airport_id'], how='left')
flight_df



#Podstawowe statystyki dotyczące temperatury maksymalnej

# Wyznacz ilość
count_tmax = round(flight_df['tmax'].count(),2)

#średnia
mean_tmax = round(flight_df['tmax'].mean(),2)

#mediana
median_tmax = round(flight_df['tmax'].median(),2)

#odchylenie
std_tmax = round(flight_df['tmax'].std(),2)

#min
min_tmax = round(flight_df['tmax'].min(), 2)

#max
max_tmax = round(flight_df['tmax'].max(), 2)


#percentyle
percentiles = [0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]
perc_tmax = round(flight_df['tmax'].quantile([0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]), 2)

flight_tmax_analysis_df = pd.DataFrame({
    'Statistics': ['count', 'mean', 'median', 'std', 'min', 'max'] + [f'percentile {p*100}%' for p in percentiles],
    'Value': [count_tmax, mean_tmax, median_tmax, std_tmax, min_tmax, max_tmax] + list(perc_tmax)
})

#Obliczenie odsetka opóźnień lotów dla każdej wartości temperatury maksymalnej
delay_percentage = flight_df.groupby('tmax')['is_delayed'].mean().reset_index()

flight_df = flight_df[flight_df['tmax'] >= 0]

#Obliczenie odsetka opóźnień lotów dla każdej wartości temperatury maksymalnej po oczyszczeniu danych
delay_percentage_v2 = flight_df.groupby('tmax')['is_delayed'].mean().reset_index()

#Wizualizacja 
plt.figure(figsize=(10, 6))
plt.bar(delay_percentage_v2['tmax'], delay_percentage_v2['is_delayed'])
plt.title('Delay Percentage vs. Max Temperature', fontweight='bold')
plt.xlabel('Max Temperature (°F)')
plt.ylabel('Delay Percentage')
plt.grid(axis='y')
plt.xlim(left=0)
plt.show()

#Dokonanie agregacji tmax co 5 stopni
flight_df['tmax'] = flight_df['tmax'].astype(int)
bin_width = 5
max_tmax = flight_df['tmax'].max()

#Tworzenie zakresów dla tmax
custom_bins = list(range(0, max_tmax + bin_width, bin_width))

#Agregacja danych
flight_df['tmax_range'] = pd.cut(flight_df['tmax'], bins=custom_bins, labels=[f'{x}-{x+bin_width-1}' for x in custom_bins[:-1]], ordered=False)

#Obliczenie odsetka opóźnień lotów dla każdego zakresu temperatury maksymalnej
delay_percentage_agg = flight_df.groupby('tmax_range')['is_delayed'].mean().reset_index()

# Wizualizacja 
plt.figure(figsize=(15, 10))
plt.bar(delay_percentage_agg['tmax_range'], delay_percentage_agg['is_delayed'])
plt.title('Delay Percentage vs. Max Temperature', fontweight='bold')
plt.xlabel('Max Temperature (°F)')
plt.ylabel('Delay Percentage')
plt.grid(axis='y')
plt.xticks(rotation=45)
plt.show()